# Загрузка и предобработка данных

В этом ноутбуке выполняется загрузка исторических данных OHLCV, вычисление log_returns и подготовка базового датафрейма.  

**Ключевые задачи:**
- Загрузка для всех нужных тикеров MOEX
- Чистка пропусков и сплит-дат
- Вычисление log_returns без корректировки по дивидендам/корп. действиям (по решению)
- Сохранение промежуточного результата для Feature Engineering


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


✅ Библиотеки загружены


In [2]:
# Определение пути к данным
current_dir = Path.cwd()
DATA_DIR = current_dir / "data" / "MOEX_DATA"

# Поиск данных в альтернативных местах
if not DATA_DIR.exists():
    alt_path = current_dir.parent / "ML" / "data" / "MOEX_DATA"
    if alt_path.exists():
        DATA_DIR = alt_path
    else:
        for parent in [current_dir, current_dir.parent]:
            test_path = parent / "data" / "MOEX_DATA"
            if test_path.exists():
                DATA_DIR = test_path
                break

print(f"Путь к данным: {DATA_DIR}")
print(f"Найдено: {DATA_DIR.exists()}")


Путь к данным: e:\Python\VolatilityChecker\MOEXScanner\ML\data\MOEX_DATA
Найдено: True


## Функции загрузки и расчета


In [3]:
def calculate_log_returns(prices):
    """Вычисляет логарифмические доходности: log(P_t / P_{t-1})"""
    if not isinstance(prices, pd.Series):
        prices = pd.Series(prices)
    log_prices = np.log(prices)
    return log_prices - log_prices.shift(1)


def load_ticker_data(ticker, data_dir=DATA_DIR, timeframe='1D'):
    """Загружает данные по тикеру из CSV"""
    ticker_dir = data_dir / ticker / timeframe
    if not ticker_dir.exists():
        raise FileNotFoundError(f"Папка {ticker_dir} не найдена")
    
    csv_files = list(ticker_dir.glob("*.csv"))
    if not csv_files:
        raise FileNotFoundError(f"CSV файл не найден в {ticker_dir}")
    
    df = pd.read_csv(csv_files[0])
    
    # Преобразование дат
    if 'end' in df.columns:
        df['date'] = pd.to_datetime(df['end'])
    elif 'begin' in df.columns:
        df['date'] = pd.to_datetime(df['begin'])
    
    df = df.sort_values('date').reset_index(drop=True)
    return df

print("✅ Функции загружены")


✅ Функции загружены


## Тестирование на одном тикере


In [4]:
# Тестирование на одном тикере
ticker = 'SBER'
df = load_ticker_data(ticker, timeframe='1D')
df['log_return'] = calculate_log_returns(df['close'])

print(f"\nТикер: {ticker}")
print(f"Период: {df['date'].min().date()} → {df['date'].max().date()}")
print(f"Записей: {len(df)}")
print(f"\nСтатистика log returns:")
print(df['log_return'].describe())



Тикер: SBER
Период: 2020-10-12 → 2025-10-11
Записей: 1302

Статистика log returns:
count    1301.000000
mean        0.000247
std         0.023760
min        -0.455918
25%        -0.007616
50%         0.000539
75%         0.009146
max         0.117686
Name: log_return, dtype: float64


## Загрузка нескольких тикеров


In [5]:
# Загрузка нескольких тикеров
test_tickers = ['SBER', 'GAZP', 'LKOH', 'MTSS', 'YNDX', 'ROSN', 'GMKN']
results = {}

for ticker in test_tickers:
    try:
        df = load_ticker_data(ticker, timeframe='1D')
        df['log_return'] = calculate_log_returns(df['close'])
        
        # Чистка пропусков
        df = df.dropna(subset=['close', 'log_return'])
        
        results[ticker] = df
        print(f"✅ {ticker}: {len(df)} записей")
    except Exception as e:
        print(f"❌ {ticker}: {e}")

# Сводная таблица
if results:
    summary = pd.DataFrame({
        'Тикер': list(results.keys()),
        'Записей': [len(results[k]) for k in results.keys()],
        'Среднее': [results[k]['log_return'].mean() for k in results.keys()],
        'Ст.откл.': [results[k]['log_return'].std() for k in results.keys()],
    })
    print(f"\n{summary.to_string(index=False)}")


✅ SBER: 1301 записей
✅ GAZP: 1301 записей
✅ LKOH: 1301 записей
✅ MTSS: 1300 записей
✅ YNDX: 356 записей
✅ ROSN: 1301 записей
✅ GMKN: 1297 записей

Тикер  Записей   Среднее  Ст.откл.
 SBER     1301  0.000247  0.023760
 GAZP     1301 -0.000293  0.026512
 LKOH     1301  0.000228  0.019893
 MTSS     1300 -0.000382  0.020580
 YNDX      356 -0.000228  0.019620
 ROSN     1301  0.000045  0.023672
 GMKN     1297 -0.000337  0.019813


## Сохранение результатов

Сохраняем промежуточные данные для следующих этапов Feature Engineering


In [6]:
# Сохранение промежуточного результата
OUTPUT_DIR = Path('data') / 'processed'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

for ticker, df in results.items():
    output_path = OUTPUT_DIR / f"{ticker}_ohlcv_returns.parquet"
    df.to_parquet(output_path, index=False)
    print(f"✅ Сохранено: {output_path}")

print(f"\n📁 Все данные сохранены в {OUTPUT_DIR}")


✅ Сохранено: data\processed\SBER_ohlcv_returns.parquet
✅ Сохранено: data\processed\GAZP_ohlcv_returns.parquet
✅ Сохранено: data\processed\LKOH_ohlcv_returns.parquet
✅ Сохранено: data\processed\MTSS_ohlcv_returns.parquet
✅ Сохранено: data\processed\YNDX_ohlcv_returns.parquet
✅ Сохранено: data\processed\ROSN_ohlcv_returns.parquet
✅ Сохранено: data\processed\GMKN_ohlcv_returns.parquet

📁 Все данные сохранены в data\processed
